In [86]:
import os
import subprocess
import pickle

In [87]:
# conversion factor for 1 hatree to kcals/mol
au = 627.5095 
top = "tt"
file = "a_"
vars = [ "D3A1", "D3A2", "D3S8", "!"]
change_params = [0.2885845358427619,4.505426185357342,2.9935113319271824]
struct_path = "workflow_test"
functional = "blyp"
basis = "def2-svpd"
# Dictionary of interaction energies 
inter_energies = dict({ 
    'x01': -4.982,
    'x02': -5.666,
    'x03': -6.986,
    'x04': -8.183,
    'x05': -5.822,
    'x06': -7.617,
    'x07': -8.307,
    'x08': -5.064,
    'x09': -3.087,
    'x10': -4.184,
    'x11': -5.436,
    'x12': -7.359,
    'x13': -6.251,
    'x14': -7.516,
    'x15': -8.689,
    'x16': -5.180,
    'x17': -17.407,
    'x18': -6.927,
    'x19': -7.467,
    'x20': -19.7361,
    'x21': -16.474,
    'x22': -19.736,
    'x23': -19.420,
    'x24': -2.685,
    'x25': -3.751,
    'x26': -9.672,
    'x27': -3.300,
    'x28': -5.517,
    'x29': -6.629,
    'x30': -1.358,
    'x31': -3.291,
    'x32': -3.651,
    'x33': -1.779,
    'x34': -3.741,
    'x35': -2.582,
    'x36': -1.745,
    'x37': -2.376,
    'x38': -2.967,
    'x39': -3.488,
    'x40': -2.824,
    'x41': -4.761,
    'x42': -4.052,
    'x43': -3.652,
    'x44': -1.973,
    'x45': -1.696,
    'x46': -4.215,
    'x47': -2.801,
    'x48': -3.472,
    'x49': -3.260,
    'x50': -2.828,
    'x51': -1.519,
    'x52': -4.691,
    'x53': -4.376,
    'x54': -3.267,
    'x55': -4.139,
    'x56': -3.174,
    'x57': -5.222,
    'x58': -4.189,
    'x59': -2.905,
    'x60': -4.917,
    'x61': -2.876,
    'x62': -3.491,
    'x63': -3.709,
    'x64': -2.967,
    'x65': -4.064,
    'x66': -3.930
})


In [88]:
# function to return list of parameters to swap out
def get_params(file):
    file_list = file.split()
    return [file_list[file_list.index(y) + 1] for y in vars]

# function to create dictionary to hold structures
def dict_maker(path):
    struct_dict = dict()
    file_list = [x for x in sorted(os.listdir(path)) if ".xyz" in x]
    
    for file in file_list:
        key = (file.replace(".xyz", "")
               .replace("dd", "")
               .replace("m1","")
               .replace("m2", "")
               .replace("a_","")
               .replace("c_", ""))
        if key in struct_dict.keys():
            struct_dict[key].append(f"{path}/{file}")
        else:
            struct_dict.update({key:[f"{path}/{file}"]})
            
    return struct_dict

# function to change top orca input file
def top_change(top_file, change_data):
    with open(top_file, "w") as t:
        t.write(change_data)

In [90]:
with open("/home/riley/Dispersion-Parameter-Optimization-D-POp-/tt", "r") as t:
    data = t.read()

params = get_params(data)
params

['0.2885845358427619', '4.505426185357342', '2.9935113319271824', 'blyp']

In [4]:
struct_dict = dict_maker(struct_path)
struct_dict

{'x05': ['workflow_test/x05dd.xyz',
  'workflow_test/x05m1.xyz',
  'workflow_test/x05m2.xyz']}

In [30]:
with open(f"base_calculations_{functional}_{basis}.pkl", 'rb') as f:
    base_calculations = pickle.load(f)
base_calculations

{'x01': -4.042673689348837,
 'x02': -4.289575976206804,
 'x03': -6.502826826439239,
 'x04': -6.096868564547438,
 'x05': -4.055173740101212,
 'x06': -5.989731530790744,
 'x07': -5.426960365505868,
 'x08': -3.826956461664057,
 'x09': -1.0961609276650222,
 'x10': -1.8010170315040956,
 'x11': -1.420813022692698,
 'x12': -6.298344312311194,
 'x13': -3.257516353834641,
 'x14': -4.5623108995532276,
 'x15': -4.164993100909815,
 'x16': -3.466735148378361,
 'x17': -13.456284668082485,
 'x18': -6.313493538117598,
 'x19': -6.17062058441332,
 'x20': -16.221801730316297,
 'x21': -13.111983784761787,
 'x22': -16.15851160716406,
 'x23': -15.669346823117252,
 'x24': 3.403779424434314,
 'x25': 2.883857998254184,
 'x26': 2.2347470761203017,
 'x27': 3.1130435627453674,
 'x28': 3.7963890318154427,
 'x29': 2.5922668159545053,
 'x30': 2.296002883677245,
 'x31': 1.7130299937803475,
 'x32': 0.6709608286750932,
 'x33': 2.0422639222099193,
 'x34': 3.1513625822518665,
 'x35': 1.7492737031919838,
 'x36': 0.9884159

In [27]:
def energy(change_params, base_calculations, struct_dict, functional):
    
    with open(f"{top}", "r") as f:
        data = f.read()

    change = [str(y) for y in change_params]
    change.append(functional)

    params = get_params(data)
    change_data = (data.replace(params[0], change[0])
                    .replace(params[1], change[1])
                    .replace(params[2], change[2])
                    .replace(params[3], change[3]))

    energy_list = []
    strd_values = []

    for struct in struct_dict.keys():
        energies = []
        strd_values.append(inter_energies[struct])

        for item in struct_dict[struct]:

            if "a_" in item:
                change_data = data.replace("xyz 0", "xyz -1")
            elif "c_" in item:
                change_data = data.replace("xyz 0", "xyz 1")

            top_change(top, change_data)
            subprocess.run(f"cat {top} {item} bb > input", shell=True)
            energies.append(float(subprocess.check_output(f"/home/riley/bin/orca_6_0_0_shared_openmpi416/orca input| grep 'Dispersion'| awk '{{print $3}}' ORS=' '",
                                                              shell=True,
                                                              executable="/bin/bash")
                                                              .decode()))
            subprocess.run("rm input*", shell=True)
            top_change(top, data)
            
        energy_list.append(base_calculations[struct] + au*(energies[0] - (energies[1]+energies[2])))

    return strd_values, energy_list

In [22]:
with open(f"{top}", "r") as f:
        data = f.read()

change = [str(y) for y in change_params]
change.append(functional)

params = get_params(data)
change_data = (data.replace(params[0], change[0])
                .replace(params[1], change[1])
                .replace(params[2], change[2])
                .replace(params[3], change[3]))

energy_list = []
strd_values = []

for struct in struct_dict.keys():
    energies = []
    strd_values.append(inter_energies[struct])
    
    for item in struct_dict[struct]:

        if "a_" in item:
            change_data = data.replace("xyz 0", "xyz -1")
        elif "c_" in item:
            change_data = data.replace("xyz 0", "xyz 1")
        
        top_change(top, change_data)
        subprocess.run(f"cat {top} {item} bb > input", shell=True)
        energies.append(float(subprocess.check_output(f"/home/riley/bin/orca_6_0_0_shared_openmpi416/orca input| grep 'Dispersion'| awk '{{print $3}}' ORS=' '",
                                                          shell=True,
                                                          executable="/bin/bash")
                                                          .decode()))
        subprocess.run("rm input*", shell=True)
        top_change(top, data)
    energy_list.append(base_calculations[struct] + au*(energies[0] - (energies[1]+energies[2])))
        
params

['1.5340745924210992', '2.2986916469230967', '4.8958037257994045', 'pbe']

In [34]:
strd, en_list = energy(change_params, base_calculations, struct_dict, functional)
strd

[-5.822]

In [35]:
en_list

[-5.7014104003287125]

In [36]:
bases = [x for x in sorted(os.listdir(os.getcwd())) if "base_calculations" in x]
bases

['base_calculations_blyp_def2-svp.pkl',
 'base_calculations_blyp_def2-svpd.pkl',
 'base_calculations_blyp_def2-tzvp.pkl',
 'base_calculations_pbe_def2-svp.pkl',
 'base_calculations_pbe_def2-svpd.pkl',
 'base_calculations_pbe_def2-tzvp.pkl',
 'base_calculations_tpss_def2-svp.pkl',
 'base_calculations_tpss_def2-svpd.pkl',
 'base_calculations_tpss_def2-tzvp.pkl']

In [41]:
for key in inter_energies.keys():
    with open("s66_ref_energies", 'a') as bb:
        bb.write(f"{key} {inter_energies[key]}\n")

In [47]:
import re

In [84]:
def value_dict_maker(path):
    with open(path, "r") as f:
        value_list = [ x.split() for x in f.read().split("\n") if x]
    return {x[0]: float(x[1]) for x in value_list}

In [85]:
base = value_dict_maker("s66_ref_energies")
base

{'x01': -4.982,
 'x02': -5.666,
 'x03': -6.986,
 'x04': -8.183,
 'x05': -5.822,
 'x06': -7.617,
 'x07': -8.307,
 'x08': -5.064,
 'x09': -3.087,
 'x10': -4.184,
 'x11': -5.436,
 'x12': -7.359,
 'x13': -6.251,
 'x14': -7.516,
 'x15': -8.689,
 'x16': -5.18,
 'x17': -17.407,
 'x18': -6.927,
 'x19': -7.467,
 'x20': -19.7361,
 'x21': -16.474,
 'x22': -19.736,
 'x23': -19.42,
 'x24': -2.685,
 'x25': -3.751,
 'x26': -9.672,
 'x27': -3.3,
 'x28': -5.517,
 'x29': -6.629,
 'x30': -1.358,
 'x31': -3.291,
 'x32': -3.651,
 'x33': -1.779,
 'x34': -3.741,
 'x35': -2.582,
 'x36': -1.745,
 'x37': -2.376,
 'x38': -2.967,
 'x39': -3.488,
 'x40': -2.824,
 'x41': -4.761,
 'x42': -4.052,
 'x43': -3.652,
 'x44': -1.973,
 'x45': -1.696,
 'x46': -4.215,
 'x47': -2.801,
 'x48': -3.472,
 'x49': -3.26,
 'x50': -2.828,
 'x51': -1.519,
 'x52': -4.691,
 'x53': -4.376,
 'x54': -3.267,
 'x55': -4.139,
 'x56': -3.174,
 'x57': -5.222,
 'x58': -4.189,
 'x59': -2.905,
 'x60': -4.917,
 'x61': -2.876,
 'x62': -3.491,
 'x63':

In [76]:
with open("s66_ref_energies", "r") as f:
    d = f.read()
d

'x01 -4.982\nx02 -5.666\nx03 -6.986\nx04 -8.183\nx05 -5.822\nx06 -7.617\nx07 -8.307\nx08 -5.064\nx09 -3.087\nx10 -4.184\nx11 -5.436\nx12 -7.359\nx13 -6.251\nx14 -7.516\nx15 -8.689\nx16 -5.18\nx17 -17.407\nx18 -6.927\nx19 -7.467\nx20 -19.7361\nx21 -16.474\nx22 -19.736\nx23 -19.42\nx24 -2.685\nx25 -3.751\nx26 -9.672\nx27 -3.3\nx28 -5.517\nx29 -6.629\nx30 -1.358\nx31 -3.291\nx32 -3.651\nx33 -1.779\nx34 -3.741\nx35 -2.582\nx36 -1.745\nx37 -2.376\nx38 -2.967\nx39 -3.488\nx40 -2.824\nx41 -4.761\nx42 -4.052\nx43 -3.652\nx44 -1.973\nx45 -1.696\nx46 -4.215\nx47 -2.801\nx48 -3.472\nx49 -3.26\nx50 -2.828\nx51 -1.519\nx52 -4.691\nx53 -4.376\nx54 -3.267\nx55 -4.139\nx56 -3.174\nx57 -5.222\nx58 -4.189\nx59 -2.905\nx60 -4.917\nx61 -2.876\nx62 -3.491\nx63 -3.709\nx64 -2.967\nx65 -4.064\nx66 -3.93\n'

In [4]:
a = "False"
bool(a)

True

In [3]:
if not bool(a):
    print("a")

In [5]:
with open("b", "w") as tt:
    tt.write("hi")